In [2]:
import pandas as pd
import numpy as np
import math
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')

from fbprophet import Prophet
from pyramid.arima import auto_arima

from sklearn.metrics import mean_absolute_error


def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))

# tsfresh functions
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

In [3]:
def plotting_features(exog, mode):
    """
    plotting features
    """
    
    if mode == 'prophet':
        Exog = exog.values
        
    elif mode == 'pyramid':
        Exog = exog

    else:
        print("check mode")
        return None
    
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    
    fig.subplots_adjust(top=2, bottom=1,  right=2, left=0, wspace=0.6)
    

    ax1.plot(Time,Exog[:,0])
    ax1.set_ylabel('exog_1', bbox=box)


    ax3.set_ylabel('exog_2',bbox=box)
    ax3.plot(Time,Exog[:,1])

    labelx = -0.3  # axes coords

    ax2.plot(Time,Exog[:,2])
    ax2.set_ylabel('exog_3', bbox=box)
    ax2.yaxis.set_label_coords(labelx, 0.5)


    ax4.plot(Time,Exog[:,3])
    ax4.set_ylabel('exog_4', bbox=box)
    ax4.yaxis.set_label_coords(labelx, 0.5)

    plt.show()
    
    pass

In [4]:
def plot_results(time_series, Time, train_prediction, test_prediction, y_test, mode, description):
    
    """
    Plots the original time series and it prediction
    """
    if mode == 'prophet':
        ts = time_series["y"].values
        train_pr = train_prediction["yhat"].values
        test_pr = test_prediction["yhat"].values
    
    elif mode == 'pyramid':
        train_pr = train_prediction
        test_pr = test_prediction
        ts = time_series
        
    
    else:
        print("Check mode")
        return None
    
    plt.figure(figsize=(10,7))

    plt.plot(Time, ts, label = "true")
    
    plt.plot(Time[start_point+1: end_point], np.concatenate([train_pr, test_pr])[1:], 
         label = "predictions, \n MAPE = {0}, MAE = {1}".format(
             np.round(mean_absolute_percentage_error(test_pr, y_test), 3),
             np.round(mean_absolute_error(test_pr, y_test), 3)))


    plt.axvline(x=middle_point, label = "train-test-split", color = 'r')

    plt.xlabel("time", size = 20)
    plt.ylabel("value", size = 20)
    plt.title("{}".format(description), size = 20)


    plt.legend(fontsize = 15)

    plt.show()
    pass

In [142]:
def fit_pyramid(data_train, exog_train = None, params = None):

    """
    pyramid_mode {"stepwise", "random_search"}

    """
    
    if params['pyramid_mode'] == "stepwise":
        model = auto_arima(data_train, exogenous=exog_train, start_p=0, start_q=0, max_p=5, max_q=5,
        m=params['period'], n_jobs = params['n_jobs'], scoring = params['scoring'], 
        out_of_sample_size = params['out_of_sample_size'], 
        start_P=0, start_Q = 0, max_d=2, max_D=2,max_P = 5, max_Q = 5, trace=True,
        error_action='ignore', information_criterion = params['ic'], trend = params['trend'],
        suppress_warnings=True, stepwise=True)
    
    elif params['pyramid_mode'] == 'random_search':
        model = auto_arima(data_train, exogenous=exog_train, start_p=0, start_q=0, max_p=5, max_q=5, 
        m=params['period'], scoring = params['scoring'], out_of_sample_size = params['out_of_sample_size'],
        start_P=0, start_Q = 0, n_jobs=params['n_jobs'], max_d=2, max_D=2,max_P = 5, max_Q = 5, trace=True,
        error_action='ignore', information_criterion = params['ic'], trend = params['trend'],
        suppress_warnings=True, 
        stepwise=False, random=True, random_state=params['random_state'], n_fits = params['n_fits'])
    
    return model

In [143]:
def fit_rolling_pyramid(y_train, max_timeshift = 10, rolling_direction = 1, params = None):

    df_shift, y = make_forecasting_frame(y_train, kind = "price", 
                        max_timeshift = max_timeshift, rolling_direction = rolling_direction)
    X_train = extract_features(df_shift, column_id="id", column_sort="time", 
                                    column_value="value", impute_function=impute, show_warnings=False)    
    X_train.dropna(axis = 1, inplace = True)
    X_train = np.array(X_train)
    ts = y_train[2:]
    exog = X_train[:-1]
    last_exog = X_train[-1]
    model = fit_pyramid(data_train=ts, exog_train=exog,params = params)
    predict_in_sample = model.predict_in_sample(exogenous = exog)
    return model, last_exog, predict_in_sample

In [138]:
 def predict_rolling_pyramid(model, last_exog, y_train, forecast_horizont, max_timeshift = 10, rolling_direction = 1):
        
        """
        Predicting values on the next forecast_horizont values
        """
        predictions = np.empty(forecast_horizont)
        predictions[0] = model.predict(n_periods = 1, exogenous = last_exog.reshape(1,-1))
        for it in range(1,forecast_horizont):
            
            y_train_tmp = np.append(y_train, predictions[it-1])
            df_shift, y = make_forecasting_frame(y_train_tmp, kind = "price", 
                            max_timeshift = max_timeshift, rolling_direction = rolling_direction)
        
            X_train = extract_features(df_shift, column_id="id", column_sort="time", 
                    column_value="value", impute_function=impute, show_warnings=False)
        
            X_train.dropna(axis = 1, inplace = True)
            X_train = np.array(X_train)
            ts_tmp =y_train_tmp[2:]
            exog_tmp = X_train[:-1]
            last_exog = X_train[-1].reshape(1,-1)
            
            model = model.fit( y = ts_tmp, exogenous = exog_tmp, enforce_invertibility = True)
            y_pred = model.predict(n_periods = 1, exogenous = last_exog)
            predictions[it] = y_pred
        
        return predictions

In [112]:
df = pd.read_csv('AirPassengers.csv', usecols=[1])

In [113]:
df.shape

(144, 1)

In [114]:
data_train = np.array(df.iloc[:120]).squeeze()
data_test = np.array(df[120:]).squeeze()

In [140]:
params = {}
params['pyramid_mode'] = 'random_search'
params['ic'] = 'aic'
params['period'] = 12
params['random_state'] = 42
params['n_fits'] = 10
params['trend'] = 'c'
params['n_jobs'] = 1
params['scoring'] = 'mae'
params['out_of_sample_size'] = 0
params['dynamic'] = False

In [141]:
model, last, predict_in_sample = fit_rolling_pyramid(y_train=data_train, params=params)

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/settings.py:106: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if callable(func) and hasattr(func, "fctype") and len(getargspec(func).args) == 1:

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.54it/s

In [144]:
model.summary()

/home/nurlan16/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  118
Model:             SARIMAX(0, 0, 2)x(1, 0, 0, 12)   Log Likelihood                2065.182
Date:                            Mon, 23 Jul 2018   AIC                          -2532.364
Time:                                    18:14:19   BIC                           -318.587
Sample:                                         0   HQIC                         -1633.505
                                            - 118                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -2.506e-09   1.41e-12  -1779.009      0.000   -2.51e-09    -2.5e-09
x1            -0.0002   3.29e-13  -7.57e+08      0.000      -0.000      -0.000
x2            -0.3943   1.93e-10  -2.05e+09      0.000      -0.394      -0.394
x3            -0.2996        nan        nan        nan         nan         nan
x4            -0.9298   1.98e-12   -4.7e+11      0.000      -0.930      -0.930
x5            -0.6383   2.72e-12  -2.34e+11      0.000      -0.638      -0.638
const       1.288e-08        nan        nan        nan         nan         nan
x6          3.263e-08   1.03e-12   3.17e+04      0.000    3.26e-08    3.26e-08
x7          1.198e-08        nan        nan        nan         nan         nan
x8          9.073e-09        nan        nan        nan         nan         nan
x9         -8.246e-11        nan        nan        nan         nan         nan
x10         4.132e-12        nan        nan        nan         nan         nan
x11         1.281e-10        nan        nan        nan         nan         nan
x12        -1.145e-10   2.96e-14  -3867.163      0.000   -1.15e-10   -1.14e-10
x13           -0.4547   1.11e-10  -4.11e+09      0.000      -0.455      -0.455
x14           -1.6084   1.95e-11  -8.26e+10      0.000      -1.608      -1.608
x15           -1.8798   2.01e-10  -9.36e+09      0.000      -1.880      -1.880
x16        -2.129e-11        nan        nan        nan         nan         nan
x17         5.379e-12   3.91e-22   1.38e+10      0.000    5.38e-12    5.38e-12
x18        -8.586e-12   1.07e-21  -7.99e+09      0.000   -8.59e-12   -8.59e-12
x19          3.56e-12        nan        nan        nan         nan         nan
x20         8.675e-12   7.84e-22   1.11e+10      0.000    8.67e-12    8.67e-12
x21         1.869e-13        nan        nan        nan         nan         nan
x22         -1.32e-12        nan        nan        nan         nan         nan
x23         3.694e-12   1.61e-22    2.3e+10      0.000    3.69e-12    3.69e-12
x24        -1.286e-11        nan        nan        nan         nan         nan
x25            0.0421   1.95e-11   2.16e+09      0.000       0.042       0.042
x26            1.4509    2.2e-11   6.61e+10      0.000       1.451       1.451
x27            0.0397   2.38e-11   1.67e+09      0.000       0.040       0.040
x28         6.656e-12        nan        nan        nan         nan         nan
x29         2.315e-12        nan        nan        nan         nan         nan
x30         9.124e-12   1.63e-22   5.61e+10      0.000    9.12e-12    9.12e-12
x31        -6.956e-12   4.35e-23   -1.6e+11      0.000   -6.96e-12   -6.96e-12
x32        -1.717e-12        nan        nan        nan         nan         nan
x33        -2.309e-12        nan        nan        nan         nan         nan
x34        -7.518e-13        nan        nan        nan         nan         nan
x35        -1.404e-13        nan        nan        n

In [145]:
predictions = predict_rolling_pyramid(model,last,data_train,24)

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/settings.py:106: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if callable(func) and hasattr(func, "fctype") and len(getargspec(func).args) == 1:

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.61it/s

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/settings.py:106: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if callable(func) and hasattr(func, "fctype") and len(getargspec(func).args) == 1:

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.66it/s

ValueError: Non-stationary starting autoregressive parameters found with `enforce_stationarity` set to True.

In [126]:
predictions

array([231.73185279, 190.8826595 ])

In [139]:
predictions = predict_rolling_pyramid(model,last,data_train,3)

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/settings.py:106: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if callable(func) and hasattr(func, "fctype") and len(getargspec(func).args) == 1:

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.62it/s

/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  grouped_data = df.groupby(grouper)
/home/nurlan16/anaconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/settings.py:106: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if callable(func) and hasattr(func, "fctype") and len(getargspec(func).args) == 1:

Feature Extraction: 100%|██████████| 10/10 [00:05<00:00,  1.74it/s

ValueError: non-invertible starting seasonal moving average parameters found with `enforce_invertibility` set to True.

In [133]:
predictions

array([231.7318528, 190.8826595])